In [1]:
import torch
import pandas as pd
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch.nn as nn
from PIL import Image
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) Rebuild your B0 classifier exactly as before
region_model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
num_ftrs = region_model.classifier[1].in_features  # 1280
region_model.classifier = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(num_ftrs, 512),
    nn.ReLU(inplace=True),
    nn.Linear(512,     256),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(256,     15)
)

# 2) Load your saved B0 weights
region_model.load_state_dict(torch.load("best_model.pth"))
region_model = region_model.to(device).eval()

# 3) Define inference transforms (same as you used when saving)
region_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# 4) Compute and save priors as before…
df_train = pd.read_csv("labels_train.csv")
df_val   = pd.read_csv("labels_val.csv")

def compute_priors(df, img_dir):
    priors = []
    for fn in df["filename"]:
        img = Image.open(Path(img_dir)/fn).convert("RGB")
        x = region_transforms(img).unsqueeze(0).to(device)
        with torch.no_grad():
            logits = region_model(x)
        priors.append(int(logits.argmax(1).item()))
    return priors

df_train["region_prior"] = compute_priors(df_train, "images_train")
df_val["region_prior"]   = compute_priors(df_val,   "images_val")

df_train.to_csv("labels_train_with_prior.csv", index=False)
df_val.to_csv("labels_val_with_prior.csv",   index=False)

print("Extended CSVs saved with region_prior column.")


Extended CSVs saved with region_prior column.


In [4]:
# Cell 2: Updated Dataset that returns (img, region_prior, target)

import math
import pandas as pd
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# re-use your existing transforms
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(256, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.3,0.3,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

class DirectionWithPriorDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df       = pd.read_csv(csv_file)
        self.img_dir  = Path(img_dir)
        self.transform= transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row    = self.df.iloc[idx]
        img    = Image.open(self.img_dir/row["filename"]).convert("RGB")
        θ      = float(row["angle"])
        r      = math.radians(θ)
        target = torch.tensor([math.sin(r), math.cos(r)], dtype=torch.float32)
        prior  = torch.tensor(int(row["region_prior"]), dtype=torch.long)
        if self.transform:
            img = self.transform(img)
        return img, prior, target

# now point to your _with_prior CSVs
train_ds = DirectionWithPriorDataset("labels_train_with_prior.csv",
                                     "images_train", train_transforms)
val_ds   = DirectionWithPriorDataset("labels_val_with_prior.csv",
                                     "images_val",   val_transforms)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False,
                          num_workers=4, pin_memory=True)


In [5]:
# Cell 6: Model that consumes image + region_prior → sin/cos

import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

class DirectionWithPriorModel(nn.Module):
    def __init__(self, num_regions=15, emb_dim=32):
        super().__init__()
        # 1) EfficientNet-B0 backbone (no head)
        weights = EfficientNet_B0_Weights.IMAGENET1K_V1
        self.backbone = efficientnet_b0(weights=weights)
        in_f = self.backbone.classifier[1].in_features  # 1280
        self.backbone.classifier = nn.Identity()
        # 2) Region-ID embedding
        self.region_emb = nn.Embedding(num_regions, emb_dim)
        # 3) Regression head: (in_f + emb_dim) → 512 → 2
        self.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_f + emb_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 2)
        )

    def forward(self, img, prior):
        feat = self.backbone(img)             # [B,1280]
        emb  = self.region_emb(prior)         # [B,emb_dim]
        x    = torch.cat([feat, emb], dim=1)  # [B,1280+emb_dim]
        return self.head(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DirectionWithPriorModel().to(device)
print(model)


DirectionWithPriorModel(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
         

In [4]:
# Cell 3: Training loop (phase 1 + phase 2 as before, just unpack priors)

import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR
from tqdm import tqdm
import math

criterion = nn.MSELoss()

# Phase 1: head only
for p in model.backbone.features.parameters():
    p.requires_grad = False

optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()),
                 lr=1e-3, weight_decay=1e-4)
scheduler = OneCycleLR(optimizer, max_lr=1e-3,
    steps_per_epoch=len(train_loader), epochs=10)

def train_epoch():
    model.train()
    tot = 0
    running_loss = 0
    for imgs, priors, targets in tqdm(train_loader, desc="Train", leave=False):
        imgs, priors, targets = imgs.to(device), priors.to(device), targets.to(device)
        optimizer.zero_grad()
        outs = model(imgs, priors)
        normed = outs / outs.norm(dim=1, keepdim=True)
        loss = F.mse_loss(normed, targets)
        loss.backward(); optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        tot += imgs.size(0)
    return running_loss / tot

def validate_epoch():
    model.eval(); tot=0; run=0; mae=0
    with torch.no_grad():
        for imgs, priors, targets in tqdm(val_loader, desc="Val  ", leave=False):
            imgs, priors, targets = imgs.to(device), priors.to(device), targets.to(device)
            outs = model(imgs, priors)
            normed = outs / outs.norm(dim=1, keepdim=True)
            run += F.mse_loss(normed, targets).item() * imgs.size(0)
            sin_p, cos_p = normed[:,0], normed[:,1]
            pred_ang = (torch.atan2(sin_p, cos_p)*180/math.pi) %360
            sin_t, cos_t = targets[:,0], targets[:,1]
            true_ang = (torch.atan2(sin_t, cos_t)*180/math.pi)%360
            diff = torch.abs(pred_ang-true_ang)
            diff = torch.minimum(diff,360-diff)
            mae += diff.sum().item()
            tot += imgs.size(0)
    return run/tot, mae/tot

# run phase1
for epoch in range(1,11):
    trl = train_epoch(); vll, vmae = validate_epoch()
    scheduler.step()
    print(f"P1 E{epoch} L={trl:.4f} MAE={vmae:.2f}")

# Phase 2: unfreeze last 6 blocks + head
for blk in model.backbone.features[-6:]:
    for p in blk.parameters():
        p.requires_grad = True

optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()),
                 lr=1e-4, weight_decay=1e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=20)

for epoch in range(1,51):
    trl = train_epoch(); vll, vmae = validate_epoch()
    scheduler.step()
    print(f"P2 E{epoch} L={trl:.4f} MAE={vmae:.2f}")

torch.save(model.state_dict(), "model.pth")
print("Saved model weights to model.pth")


P1 E1 L=0.9474 MAE=83.23


P1 E2 L=0.8976 MAE=80.86


P1 E3 L=0.8661 MAE=79.03


P1 E4 L=0.8738 MAE=78.64


P1 E5 L=0.8470 MAE=77.76


...

P2 E46 L=0.1645 MAE=34.98


P2 E47 L=0.1426 MAE=34.61


P2 E48 L=0.1702 MAE=35.93


P2 E49 L=0.1457 MAE=34.83


P2 E50 L=0.1268 MAE=33.35
Saved model weights to model.pth


In [8]:
# Cell 7: Reload saved model & re-create val_loader/test_loader (kernel was reset)

import os, math, csv, pandas as pd
from pathlib import Path
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# 0) Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) Re-define your Dataset (unchanged; returns img, prior, target for val)
class DirectionWithPriorDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df        = pd.read_csv(csv_file)
        self.img_dir   = Path(img_dir)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row    = self.df.iloc[idx]
        img    = Image.open(self.img_dir/row["filename"]).convert("RGB")
        θ      = float(row["angle"])
        r      = math.radians(θ)
        target = torch.tensor([math.sin(r), math.cos(r)], dtype=torch.float32)
        prior  = torch.tensor(int(row["region_prior"]), dtype=torch.long)
        if self.transform:
            img = self.transform(img)
        return img, prior, target

# 2) Re-define your Model exactly as before
class DirectionWithPriorModel(nn.Module):
    def __init__(self, num_regions=15, emb_dim=32):
        super().__init__()
        weights = EfficientNet_B0_Weights.IMAGENET1K_V1
        backbone = efficientnet_b0(weights=weights)
        in_f = backbone.classifier[1].in_features  # 1280
        backbone.classifier = nn.Identity()
        self.backbone   = backbone
        self.region_emb = nn.Embedding(num_regions, emb_dim)
        self.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_f + emb_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 2)
        )

    def forward(self, img, prior):
        feat = self.backbone(img)             
        emb  = self.region_emb(prior)         
        x    = torch.cat([feat, emb], dim=1)  
        return self.head(x)

# 3) Instantiate & load weights
model = DirectionWithPriorModel(num_regions=15, emb_dim=32).to(device)
model.load_state_dict(torch.load("model.pth", map_location=device))
model.eval()

# 4) Re-create val_loader
val_transforms = T.Compose([
    T.Resize(256),
    T.CenterCrop(256),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_ds = DirectionWithPriorDataset("labels_val_with_prior.csv",
                                   "images_val", val_transforms)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False,
                        num_workers=4, pin_memory=True)

# 5) Build test_loader (no targets, just images)
class TestImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        suffixes = {'.jpg','.jpeg','.png','.bmp','.tif','.tiff'}
        files = sorted(os.listdir(root_dir))
        self.paths = [
            os.path.join(root_dir,f)
            for f in files
            if os.path.splitext(f)[1].lower() in suffixes
        ]
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, os.path.basename(self.paths[idx])

test_ds     = TestImageDataset("images_test", transform=val_transforms)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False,
                         num_workers=4, pin_memory=True)


In [9]:
# Cell 8: Export final CSV of val + test predictions (using region_priors from region_predictions.csv)

# 6) Load your saved region_ids (val+test) from region_predictions.csv
reg_df = pd.read_csv("region_predictions.csv")
all_region_ids = reg_df["Region_ID"].astype(int).tolist()
n_val = len(val_ds)    # 369
region_val_ids  = all_region_ids[:n_val]
region_test_ids = all_region_ids[n_val:]

# 7) Predict on VAL set
val_preds = []; offset = 0
with torch.no_grad():
    for imgs, priors, _ in val_loader:
        bs = imgs.size(0)
        imgs  = imgs.to(device)
        # override priors with the correct region IDs
        batch_ids = region_val_ids[offset:offset+bs]; offset += bs
        priors    = torch.tensor(batch_ids, dtype=torch.long, device=device) - 1

        outs   = model(imgs, priors)
        normed = outs / outs.norm(dim=1, keepdim=True)
        sin_p, cos_p = normed[:,0], normed[:,1]
        angs = (torch.atan2(sin_p, cos_p)*180/math.pi) % 360
        val_preds.extend(angs.cpu().tolist())
assert offset == n_val

# 8) Predict on TEST set
test_preds = []; offset = 0
with torch.no_grad():
    for imgs, fnames in test_loader:
        bs = imgs.size(0)
        imgs = imgs.to(device)
        batch_ids = region_test_ids[offset:offset+bs]; offset += bs
        priors    = torch.tensor(batch_ids, dtype=torch.long, device=device) - 1

        outs   = model(imgs, priors)
        normed = outs / outs.norm(dim=1, keepdim=True)
        sin_p, cos_p = normed[:,0], normed[:,1]
        angs = (torch.atan2(sin_p, cos_p)*180/math.pi) % 360
        test_preds.extend(angs.cpu().tolist())
assert offset == len(test_ds)

# 9) Write combined CSV
with open("val_and_test_with_prior_predictions.csv","w",newline="") as f:
    wr = csv.writer(f)
    wr.writerow(["id","angle"])
    for i,a in enumerate(val_preds):
        wr.writerow([i,a])
    for i,a in enumerate(test_preds, start=n_val):
        wr.writerow([i,a])

print(f"Wrote {n_val} val + {len(test_preds)} test predictions")


Wrote 369 val + 369 test predictions
